In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Загрузка данных из CSV-файла
df = pd.read_csv("G:\ОАД\НИРС\Greenhouse Plant Growth Metrics.csv")

# Удаляем ненужный столбец-идентификатор (не несет полезной информации для обучения)
df.drop('Random', axis=1, inplace=True)

# Разделяем признаки на числовые и категориальные
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(exclude=np.number).columns.tolist()

# Создаем предобработчик для числовых и категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        # Для числовых — замена пропусков на медиану
        ('num', SimpleImputer(strategy='median'), numerical_cols),
        # Для категориальных — замена пропусков на наиболее частое значение
        ('cat', SimpleImputer(strategy='most_frequent'), categorical_cols)
    ])

# Применяем предобработку и собираем DataFrame обратно
df_processed = pd.DataFrame(preprocessor.fit_transform(df), 
                           columns=numerical_cols + categorical_cols)

# Кодируем категориальную целевую переменную 'Class' в числовой формат
le = LabelEncoder()
df_processed['Class'] = le.fit_transform(df_processed['Class'])

# Разделяем данные на признаки (X) и целевую переменную (y)
X = df_processed.drop('Class', axis=1)
y = df_processed['Class']

# Масштабируем признаки с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделяем данные на обучающую и тестовую выборки (30% — тест)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, 
    test_size=0.3, 
    stratify=y,   # сохраняем пропорции классов
    random_state=42
)

# Обучаем базовую модель KNN с k=5
knn_base = KNeighborsClassifier(n_neighbors=5)
knn_base.fit(X_train, y_train)
y_pred_base = knn_base.predict(X_test)

# Выводим базовые метрики
print("Базовые метрики:")
print(classification_report(y_test, y_pred_base))
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred_base))

# Задаем сетку гиперпараметров для подбора
param_grid = {
    'n_neighbors': np.arange(1, 31),  # k от 1 до 30
    'weights': ['uniform', 'distance'],  # веса: равные или по расстоянию
    'metric': ['euclidean', 'manhattan']  # метрики расстояния
}

# Определяем стратегии кросс-валидации
strategies = {
    'StratifiedKFold': StratifiedKFold(n_splits=5),  # учитывает распределение классов
    'KFold': KFold(n_splits=5, shuffle=True)  # случайное перемешивание без учета классов
}
000000000
# Подбор гиперпараметров через GridSearchCV с кросс-валидацией StratifiedKFold
grid_search = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid, 
    cv=strategies['StratifiedKFold'],
    scoring='f1_weighted',  # метрика качества: взвешенное F1
    n_jobs=-1  # использование всех доступных ядер
)
grid_search.fit(X_train, y_train)

# Подбор гиперпараметров через RandomizedSearchCV с кросс-валидацией KFold
random_search = RandomizedSearchCV(
    KNeighborsClassifier(),
    param_grid,
    n_iter=20,  # количество случайных комбинаций
    cv=strategies['KFold'],
    scoring='f1_weighted',
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, y_train)

# Извлекаем лучшие модели по результатам поиска
best_knn_grid = grid_search.best_estimator_
best_knn_random = random_search.best_estimator_

# Оцениваем каждую из лучших моделей
for name, model in [('GridSearch', best_knn_grid), 
                   ('RandomizedSearch', best_knn_random)]:
    y_pred = model.predict(X_test)
    print(f"\nМетрики для {name}:")
    print(classification_report(y_test, y_pred))
    print(f"Лучшие параметры: {model.get_params()}\n")

# Сравниваем все три модели по точности и F1-метрике
models = {
    'Base KNN (k=5)': knn_base,
    'GridSearchCV': best_knn_grid,
    'RandomizedSearchCV': best_knn_random
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"{name} F1-score: {classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']:.4f}\n")


<>:13: SyntaxWarning: invalid escape sequence '\G'
<>:13: SyntaxWarning: invalid escape sequence '\G'
C:\Users\bekab\AppData\Local\Temp\ipykernel_30940\434662613.py:13: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv("G:\ОАД\НИРС\Greenhouse Plant Growth Metrics.csv")


Базовые метрики:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1500
           1       1.00      1.00      1.00      1500
           2       1.00      1.00      1.00      1500
           3       1.00      1.00      1.00      1500
           4       1.00      1.00      1.00      1500
           5       1.00      1.00      1.00      1500

    accuracy                           1.00      9000
   macro avg       1.00      1.00      1.00      9000
weighted avg       1.00      1.00      1.00      9000

Матрица ошибок:
 [[1500    0    0    0    0    0]
 [   0 1500    0    0    0    0]
 [   0    0 1500    0    0    0]
 [   0    0    0 1500    0    0]
 [   0    0    0    0 1500    0]
 [   0    0    0    0    0 1500]]

Метрики для GridSearch:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1500
           1       1.00      1.00      1.00      1500
           2       1.00      1.00   